## 4.4 Givens-Rotationen

In [ ]:
import numpy as np
from scripts.LR_Zerlegung import rueckwaerts_einsetzen

Wir implementieren die QR-Zerlegung mithilfe von Givens-Rotationen. Dabei wird die Matrix $A$ durch die Matrix $R$ überschrieben und wir erhalten als zusätzlichen Ergebnis die Matrix $Q^T$.

In [ ]:
def qr_givens(A):
    n, m = A.shape
    QT = np.identity(n, dtype=A.dtype)
    
    for i in range(m):
        for j in range(i + 1, n):
            c, s = A[i, i], -A[j, i]
            nrm = np.sqrt(c**2 + s**2)
            c, s = c / nrm, s / nrm
            for k in range(i, m):
                t1, t2 = A[i, k], A[j, k]
                A[i, k] = c * t1 - s * t2
                A[j, k] = s * t1 + c * t2
            for k in range(n):
                t1, t2 = QT[i, k], QT[j, k]
                QT[i, k] = c * t1 - s * t2
                QT[j, k] = s * t1 + c * t2
    return QT

#### Beispiel 4.20 (QR-Zerlegung nach Givens)

Wenden wir dies nun auf unser bekanntes Gleichungssystem an.

In [ ]:
A = np.array([[1,    1,    1   ],
              [0.01, 0,    0.01],
              [0,    0.01, 0.01]], dtype=np.half)
A2 = A.astype(np.single)
b = np.array([1, 0, 0.02], dtype=np.half)
x_ex = np.array([-1, 1, 1])

QT = qr_givens(A)
print(QT)

In [ ]:
np.linalg.norm(QT @ QT.T - np.eye(3, dtype=np.single), 2)

In [ ]:
QTb = np.dot(QT, b)
x = rueckwaerts_einsetzen(A, QTb)
print(x)

Dies ist nahe an der exakten Lösung $x=(-1, 1, 1)^T$, insbesondere wenn wir berücksichtigen, dass bei `half` Gleitkommadarstellung, die Maschinengenauigkeit etwa bei $\epsilon\approx 4 \times 10^{-4}$ liegt.

In [ ]:
err = np.linalg.norm(x - x_ex) / np.linalg.norm(x_ex)
print(f'||x - x_ex|| / ||x|| = {err}')

Der relative 2-Norm Fehler ist also in etwa Maschinengenauigkeit (bei `half` Gleitkommazahlen). Ähnliches beobachten wir, wenn wir $Q$ und $R$ mit einander multiplizieren und die Orthogonalität von $Q$ testen:

In [ ]:
err = np.linalg.norm(A2 - QT.transpose() @ A, ord=2) / np.linalg.norm(A2, ord=2)
print(f'||A - QR||_2 / ||A||_2 = {err}')

In [ ]:
Id = np.identity(QT.shape[0], dtype=np.single)
err = np.linalg.norm(Id - QT @ QT.T, ord=2)
print(f'||I - Q*Q^T||_2 = {err}')

*Ergänzende Einzelheiten zum Code*
- `A2` und `Id` sind als `single` gespeichert, damit wir die 2-Norm berechnen können.`